Convert RGB to incident power
===

## Diffinition

As usual, a fast visible camera generates R, G, B values at each pixel which has a specific sensitivity to the wavelength. So, To evaluate the tomographic emission profile with the physical
energy unit, we need to estimate the incident power into each pixel from the RGB value.

Each pixel data captured by the fast camera installed at PHiX, Phantom LAB 110, follows the simplified formula below:

$$
\begin{align}
D = \frac{t}{6.15\times 10^{-9} A}\int_{\mathbb{R}} \mathrm{d}\lambda\; S(\lambda)P(\lambda),
\end{align}
$$

where,

- $D$: sensor response (in digital number, 12bits, from 0-4095),
- $t$: exposure time set in camera (in second).
- $A$: the area of laser beam throwing on the sensor array ( in $\mathrm{m}^2$),
- $\lambda$: wavelength (in nm),
- $S(\lambda)$: spectral response at a specific wavelength (in A/W),
- $P(\lambda)$: the incident spectral power (in watt per nm).

Considering only Hα, Hβ and Hγ balmer-series spectral incident power $P(\lambda)$ because they dominates the PHiX experiments, we can calculate the red digital value as follows:

$$
\begin{align}
D_\text{R} & = \frac{t \times 10^9}{6.15 A}\sum_{x=\text{α, β, γ}} \int_{\varDelta\lambda_x} \mathrm{d}\lambda S_\text{R}(\lambda)P(\lambda)\\
&\approx \frac{t \times 10^9}{6.15 A}\sum_{x=\text{α, β, γ}}S_\text{R}(\lambda_x) P_x, \quad\left( \text{where }P_x \equiv \int_{\varDelta \lambda_x}\mathrm{d}\lambda P(\lambda) \right)
\end{align}
$$

where, $S_\text{R}$ is the spectral response of the red filter and $\varDelta\lambda_x$ is a wavelength range near the corresponding emission line.
The last line assumes that $S_\text{R}$ is constant with $S_\text{R}(\lambda_x)$ around the corresponding line emission.

Applying the above expression to the other color values (G, B), the following linear equation is derived:

$$
\begin{pmatrix}
D_\text{R}\\
D_\text{G}\\
D_\text{B}\\
\end{pmatrix}
=\frac{t \times 10^9}{6.15 A}
\begin{pmatrix}
S_\text{R}(\lambda_\text{α}) & S_\text{R}(\lambda_\text{β}) & S_\text{R}(\lambda_\text{γ})\\
S_\text{G}(\lambda_\text{α}) & S_\text{G}(\lambda_\text{β}) & S_\text{G}(\lambda_\text{γ})\\
S_\text{B}(\lambda_\text{α}) & S_\text{B}(\lambda_\text{β}) & S_\text{B}(\lambda_\text{γ})\\
\end{pmatrix}
\begin{pmatrix}
P_\text{α}\\
P_\text{β}\\
P_\text{γ}\\
\end{pmatrix}.
$$

Therefore, solving the above linear equation allows us to calculate each incident power.


In [ ]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from cherab.phix.observer.fast_camera.colour import (
    filter_blue,
    filter_green,
    filter_red,
    plot_RGB_filter,
)

RGB_DATA = Path().cwd().parent / "data" / "experiment" / "shot_17393_raw.npy"

Plot the RGB sensitivity with Hα, Hβ, Hγ lines
---
Let us plot RGB sensitivity data which is given by the company.

In [ ]:
# wavelength corresponding to each hydrogen balmer line
H_ALPHA = (655.6 + 656.8) * 0.5
H_BETA = (485.6 + 486.5) * 0.5
H_GAMMA = (433.6 + 434.4) * 0.5

# text greek letters
text_alpha = chr(945)
text_beta = chr(946)
text_gamma = chr(947)

In [ ]:
fig, ax = plt.subplots(dpi=150)
wavelengths = np.linspace(400, 710, 500)

# plot sensitivity
fig, ax = plot_RGB_filter(wavelengths, fig=fig, ax=ax)
ax.legend(loc="center right")

# plot hydrogen balmers
for wavelength, color, symbol in zip(
    [H_ALPHA, H_BETA, H_GAMMA], ["m", "g", "b"], [text_alpha, text_beta, text_gamma]
):
    ax.plot([wavelength, wavelength], [0, 0.2], color=color, linestyle="dashed", linewidth=0.75)
    ax.text(wavelength * 1.005, 0.16, f"H{symbol}\n{wavelength:.1f} nm", color=color)

ax.set_ylim(0, 0.19)
ax.set_xlim(wavelengths[0], wavelengths[-1]);

Convert 12bit RGB data
---
Let us convert a RGB experimental image to each balmer-series incident power.

Firstly let's show the original image data.

In [ ]:
# Load data
rgb_image = np.load(RGB_DATA)

# show image
fig = plt.figure(dpi=150)
grids = ImageGrid(fig, 111, (1, 3), cbar_mode="single", cbar_pad=0.0, axes_pad=0.02)
for i, label in enumerate(["R", "G", "B"]):
    grids[i].imshow(rgb_image[:, :, i], cmap="inferno", vmax=np.amax(rgb_image), vmin=0)
    grids[i].set_title(label)
    grids[i].set_xlabel("width")
grids[0].set_ylabel("height")
cbar = plt.colorbar(grids[0].images[-1], cax=grids.cbar_axes[0])
cbar.set_label("12bit [a.u.]")

Then, construct a spectrum $\rightarrow$ RGB converting matrix.

In [ ]:
dt = 99e-6  # [sec]
A_1px = 20e-6 * 20e-6  # [m^2]

# Spectrum [W] -> RGB digital [bit]
spectrum_to_rgb = np.array(
    [
        [filter_red(H_ALPHA), filter_red(H_BETA), filter_red(H_GAMMA)],
        [filter_green(H_ALPHA), filter_green(H_BETA), filter_green(H_GAMMA)],
        [filter_blue(H_ALPHA), filter_blue(H_BETA), filter_blue(H_GAMMA)],
    ],
    dtype=np.float64,
) * dt / (A_1px * 6.15e-9)

Solve the linear equation at each pixel.

In [ ]:
spectrum_image = np.zeros_like(rgb_image, dtype=np.float64)

for x in range(rgb_image.shape[0]):
    for y in range(rgb_image.shape[1]):
        spectrum_image[x, y, :] = np.linalg.solve(
            spectrum_to_rgb, rgb_image[x, y, :]
        )

Show the solution

In [ ]:
from matplotlib.colors import CenteredNorm

rm_text_alpha = "\\mathrm{α}"
rm_text_beta = "\\mathrm{β}"
rm_text_gamma = "\\mathrm{γ}"

fig = plt.figure(dpi=150)
grids = ImageGrid(fig, 111, (1, 3), cbar_mode="each", cbar_pad=0.0, axes_pad=0.7)
for i, symbol in enumerate([rm_text_alpha, rm_text_beta, rm_text_gamma]):
    grids[i].imshow(spectrum_image[:, :, i] / A_1px, cmap="RdBu_r", norm=CenteredNorm())
    grids[i].set_title(f"$P_{symbol}$")
    grids[i].set_xlabel("width")
    cbar = plt.colorbar(grids[i].images[0], cax=grids.cbar_axes[i])
grids[0].set_ylabel("height")
cbar.set_label("Irradiance [W/m$^2$]")

Plot each irradiance values at width 128 px.

In [ ]:
fig, ax = plt.subplots(dpi=150)
for i, color, symbol in zip(range(3), ["m", "g", "b"], [rm_text_alpha, rm_text_beta, rm_text_gamma]):
    ax.plot(spectrum_image[:, 128, i] / A_1px, color=color, label=f"$P_{symbol}$", zorder=1)

ax.axhline(0, color="k", linestyle="--", linewidth=0.7, zorder=0)
ax.set_xlim(0, 512)
ax.legend()
ax.set_xlabel("height [px]")
ax.set_ylabel("Irradiance [W/m$^2$]")
ax.set_title("at width: 128[px]");

The results show that $P_\text{γ}$ reconstruction did not work well because the negative values dominates.
However, the rest of the solutions are relatively reconstructed well.